In [1]:
# Movie recommended project
import pandas as pd
import numpy as np
import ast
movies = pd.read_csv(r"C:\Users\nikhi\CSV files\tmdb_5000_movies.csv")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [2]:
credit = pd.read_csv(r"C:\Users\nikhi\CSV files\tmdb_5000_credits.csv")
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [3]:
# Merge movie & credit dataset
movies = movies.merge(credit,on = "title")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [4]:
# Select imp coloumn 
movies = movies[['movie_id',"title","genres","cast","crew","keywords","overview"]]
movies.head(1)

,movie_id,title,genres,cast,crew,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [5]:
# Check null value
movies.isnull().sum()

movie_id    0
title       0
genres      0
cast        0
crew        0
keywords    0
overview    3
dtype: int64

In [6]:
# Drop null value
movies.dropna(inplace=True)

In [7]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [8]:
# Filter Genres & keywords
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
    
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
 

In [9]:
# Filter Cast
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
        else:
            break
    return L
    
movies['cast'] = movies['cast'].apply(convert)

In [10]:
# Filter crew (get only director name)
def fetch_d(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
    
movies['crew'] = movies['crew'].apply(fetch_d)

In [11]:
movies['overview'] = movies['overview'].apply(lambda x:x.split()) # Covert string to list 

In [12]:
# Remove spaces 
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [13]:
# Combined all 5 columns in one new column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] 

In [14]:
new_movies = movies[['movie_id','title','tags']] # Make new dataframe 
new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x)) # Convert list to string
new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower()) # Convert into lower case 

C:\Users\nikhi\AppData\Local\Temp\ipykernel_19176\2182840495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x)) # Convert list to string
C:\Users\nikhi\AppData\Local\Temp\ipykernel_19176\2182840495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower()) # Convert into lower case


In [15]:
# Convert plural words into singular like actors and actor.
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [16]:
def stem(text):
    y = []
    for i in text.split(): # Convert string to list.
        y.append(ps.stem(i))
    return " ".join(y) # Convert list to string

new_movies['tags'] = new_movies['tags'].apply(stem)

C:\Users\nikhi\AppData\Local\Temp\ipykernel_19176\2295125148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(stem)


In [17]:
# Make vector for all tags 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_movies['tags']).toarray()
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [18]:
# Calcualte distance from one vector to another
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [19]:
def recommend(movie):
    movie_index = new_movies[new_movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_movies.iloc[i[0]].title)

In [20]:
recommend('Batman Begins')

The Dark Knight
Batman
The Dark Knight Rises
Amidst the Devil's Wings
Batman


In [21]:
import pickle

In [22]:
pickle.dump(new_movies.to_dict(),open('movie_dict.pkl','wb'))


In [23]:
new_movies['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [24]:
pickle.dump(similarity,open('simirality.pkl','wb'))